In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
df1 = pd.read_csv('Bengaluru_House_Data.csv')

In [ ]:
df1

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00
...,...,...,...,...,...,...,...,...,...
13315,Built-up Area,Ready To Move,Whitefield,5 Bedroom,ArsiaEx,3453,4.0,0.0,231.00
13316,Super built-up Area,Ready To Move,Richards Town,4 BHK,NaN,3600,5.0,NaN,400.00
13317,Built-up Area,Ready To Move,Raja Rajeshwari Nagar,2 BHK,Mahla T,1141,2.0,1.0,60.00
13318,Super built-up Area,18-Jun,Padmanabhanagar,4 BHK,SollyCl,4689,4.0,1.0,488.00


In [ ]:
df1 = df1.rename(columns = {'size' : 'rooms'})
df1.dropna(subset = ['rooms'], inplace=True)
df1['bedrooms'] = df1.rooms.str.split(' ', expand=True)[0]
df1['kitchen'] = df1.rooms.str.contains('K')*1
df1['hall'] = df1.rooms.str.contains('H')*1
df1.drop('rooms', axis=1, inplace=True)
df1 = df1.astype({'bedrooms':'int64'})

In [ ]:
lower_upper_val = df1.total_sqft[df1.total_sqft.str.contains(' - ')].str.split(' - ', expand = True).astype('float64')
mean_val_sqft = lower_upper_val.assign(mean_val = (lower_upper_val[0]+lower_upper_val[1])/2).mean_val
df1.total_sqft[df1.total_sqft.str.contains(' - ')] = mean_val_sqft

<ipython-input-5-e0f10b7db24f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.total_sqft[df1.total_sqft.str.contains(' - ')] = mean_val_sqft


In [ ]:
area_conversion_to_sqft = {
    'Sq. Yards' : 9,
    'Sq. Meter' : 10.76,
    'Acres' : 43560,
    'Cents' : 435.56,
    'Guntha' : 1089,
    'Grounds' : 2400.35,
    'Perch' : 272.25
}

In [ ]:
area_to_convert = df1.total_sqft[df1.total_sqft.str.contains('[a-zA-Z]') == True]
area_values = area_to_convert.str.extract('([\d\.]+)').astype('float64')[0]
conversion_factor = df1.total_sqft.str.extract('([\D]+$)').dropna()[0].replace(area_conversion_to_sqft)
converted_area = round(area_values*conversion_factor, 2)

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13304 entries, 0 to 13319
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13304 non-null  object 
 1   availability  13304 non-null  object 
 2   location      13303 non-null  object 
 3   society       7805 non-null   object 
 4   total_sqft    13304 non-null  object 
 5   bath          13247 non-null  float64
 6   balcony       12711 non-null  float64
 7   price         13304 non-null  float64
 8   bedrooms      13304 non-null  int64  
 9   kitchen       13304 non-null  int64  
 10  hall          13304 non-null  int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 1.7+ MB


In [ ]:
df2 = df1.copy()
df2.total_sqft[converted_area.index] = converted_area
df2.total_sqft = df2.total_sqft.astype('float64').astype('int')

<ipython-input-9-51e5fb14bed9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.total_sqft[converted_area.index] = converted_area


In [ ]:
df2.dropna(subset=['bath'], inplace=True)

In [ ]:
df2.bath = df2.bath.astype('int')

In [ ]:
df2.drop(['society'],1,inplace=True)

<ipython-input-12-295d2ce3db35>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2.drop(['society'],1,inplace=True)


In [ ]:
df2.balcony.fillna(0, inplace=True)

In [ ]:
df2.rename(columns = {'total_sqft' : 'area'}, inplace=True)

In [ ]:
df2.rename(columns = {'bath' : 'bathrooms'}, inplace=True)

In [ ]:
df2.rename(columns = {'balcony' : 'numbalcony'}, inplace=True)

In [ ]:
df2['balcony'] = (df2.numbalcony > 0)

In [ ]:
df2.drop(['numbalcony'],1, inplace=True)

<ipython-input-18-0d8d973ff2c5>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2.drop(['numbalcony'],1, inplace=True)


In [ ]:
df2.info()
df2.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13247 entries, 0 to 13319
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13247 non-null  object 
 1   availability  13247 non-null  object 
 2   location      13246 non-null  object 
 3   area          13247 non-null  int64  
 4   bathrooms     13247 non-null  int64  
 5   price         13247 non-null  float64
 6   bedrooms      13247 non-null  int64  
 7   kitchen       13247 non-null  int64  
 8   hall          13247 non-null  int64  
 9   balcony       13247 non-null  bool   
dtypes: bool(1), float64(1), int64(5), object(3)
memory usage: 1.0+ MB


,area,bathrooms,price,bedrooms,kitchen,hall
count,1.324700e+04,13247.000000,13247.000000,13247.000000,13247.000000,13247.000000
mean,1.909218e+03,2.692610,112.387400,2.801917,0.810146,0.809164
std,1.732401e+04,1.341458,149.071136,1.295710,0.392201,0.392975
min,1.000000e+00,1.000000,8.000000,1.000000,0.000000,0.000000
25%,1.100000e+03,2.000000,50.000000,2.000000,1.000000,1.000000
50%,1.275000e+03,2.000000,72.000000,3.000000,1.000000,1.000000
75%,1.678500e+03,3.000000,120.000000,3.000000,1.000000,1.000000
max,1.306800e+06,40.000000,3600.000000,43.000000,1.000000,1.000000


In [ ]:
df2.drop(['availability', 'kitchen', 'hall', 'location'], 1, inplace=True)

<ipython-input-20-845cf03f02d7>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2.drop(['availability', 'kitchen', 'hall', 'location'], 1, inplace=True)


In [ ]:
df2.rename(columns = {'area_type' : 'home_type'}, inplace=True)

In [ ]:
for index, row in df2.iterrows():
  df2.at[index, 'price'] *= 100000

In [ ]:
df2['home_type'] = df2['home_type'].replace({'Super built-up  Area': 'Condos/Co-ops', 'Built-up  Area': 'Apartments', 'Plot  Area':'Houses', 'Carpet  Area':'Multi-family'})

In [ ]:
df2 = df2[df2['price'] < 100000000].reset_index(drop=True)

In [ ]:
df2.to_csv('Property_Data.csv', index=False)